# Monte Carlo Simulation for Stories/Features/Epics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

## Input your historic data
```completed``` is your historic data. Add the past 7-15 iterations (or days, sprints, etc) of data. Each item in the list should represent the number of completed stories/features/epics in that time period.

```backlog_size``` is how many items remain in your backlog.

```num_simulations``` is how many simulations you want the model to run. Higher values will likely be more accurate, with diminishing returns. 1000 is usually pretty decent, but you may want to increase this if your backlog is Very Large.

```deadline``` is for our second model, where we are forecasting how many items will be completed in so many iterations. Input your target days/weeks/sprints here.

In [ ]:
# Past Data. Each element of this list represents how many "items" were completed in each given day/sprint/timebox.
completed = [4,2,7,6,3,3,4,5,4,7,7,0,1,3,2,1,2,1,1,7]

## Start to set up our Monte Carlo Simulation
backlog_size = 50  # Items left in backlog
num_simulations = 1000  # Number of simulation runs
deadline = 15

## Run the simulations

In [ ]:
# Zero out an array with a row for each iteration of the simulation.
iterations_to_completion = np.zeros(num_simulations)

## Run our simulations
for i in range(num_simulations):
    # Initialize the backlog for this iteration
    remaining_backlog = backlog_size
    iteration_count = 0

    while remaining_backlog > 0:
        # Randomly select the number of stories completed in a sprint
        stories_completed = np.random.choice(completed)
        
        # Remove those stories from the backlog.
        remaining_backlog -= stories_completed
        iteration_count += 1

    iterations_to_completion[i] = iteration_count

iterations_to_completion = sorted(iterations_to_completion)
# iterations_to_completion now contains the number of sprints needed for each simulation run

# Initialize an array to track the total work items completed by the final iteration in each simulation
total_completed_by_final = np.zeros(num_simulations)

## Run our simulations
for sim in range(num_simulations):
    cumulative_completed = 0

    for iteration in range(deadline):
        # Randomly select the number of stories completed in a sprint
        stories_completed = np.random.choice(completed)
        
        # Update the cumulative completed count
        cumulative_completed += stories_completed

    # Store the total completed by the final iteration for this simulation
    total_completed_by_final[sim] = cumulative_completed

# Now total_completed_by_final contains the total number of work items completed by the final iteration in each simulation

print(f"{num_simulations} simulations run.")

## Display the Iterations to Completion Histogram and data table
After running you will see a histogram of the results, and a table of the granular results. The histogram shows how many iterations each run took, and therefore how likely your backlog is expected to be completed in so many runs.

In [ ]:
# Calculate histogram data
hist, bins = np.histogram(iterations_to_completion, bins=range(1, int(max(iterations_to_completion)) + 2))

# Calculate cumulative percentages
cumulative_counts = np.cumsum(hist)
total_counts = cumulative_counts[-1]
cumulative_percentages = cumulative_counts / total_counts * 100

# Displaying the cumulative distribution in a readable list format, only if the percentage is non-zero
cumulative_percentages_list = [f"{bins[i]}: {cumulative_percentages[i]:.2f}%" for i in range(len(cumulative_percentages)) if cumulative_percentages[i] > 0]

# Plotting the histogram
plt.figure(figsize=(10, 6))

# Adjusting the histogram to show only non-empty columns
non_empty_bins_indices = [i for i in range(len(hist)) if hist[i] > 0]
non_empty_bin_edges = [bins[i] for i in non_empty_bins_indices] + [bins[non_empty_bins_indices[-1] + 1]]

# Plotting the histogram with non-empty bins only
plt.hist(iterations_to_completion, bins=non_empty_bin_edges, alpha=0.7, color='blue', edgecolor='black', align='mid')

# Annotating with cumulative percentages
for i in non_empty_bins_indices:
    plt.text(bins[i], hist[i], f'{cumulative_percentages[i]:.1f}%', ha='center', va='bottom')

plt.title('Distribution of Iterations Required to Complete the Backlog with Cumulative Percentages')
plt.xlabel('Number of Iterations')
plt.ylabel('Frequency')
plt.xticks(non_empty_bins_indices)  # Set x-axis ticks to show every sprint number
plt.show()

# Printing the list
for item in cumulative_percentages_list:
    print(item)


## Display the Work Items Complete by Iteration Histogram and data table
After running you will see a histogram of the results, and a table of the granular results. The histogram shows how many work items you can expect to complete in the number of iterations you specified above in ```deadline```.

In [ ]:
# Calculate histogram data with automatic binning for the histogram
hist_auto, bins_auto = np.histogram(total_completed_by_final, bins='auto')

# Calculate histogram data with integer-aligned bins for the data table
min_value = int(min(total_completed_by_final))
max_value = int(max(total_completed_by_final)) + 1  # +1 to include the upper edge
bins_integer = np.arange(min_value, max_value + 1)  # +1 to include the last value as a bin edge
hist_integer, _ = np.histogram(total_completed_by_final, bins=bins_integer)

# Calculate cumulative percentages using the integer-aligned histogram
cumulative_counts = np.cumsum(hist_integer)
total_counts = cumulative_counts[-1]
cumulative_percentages = cumulative_counts / total_counts * 100

# Displaying the cumulative distribution in a readable list format, only if the percentage is non-zero
cumulative_percentages_list = [f"{bins_integer[i]}: {cumulative_percentages[i]:.2f}%" for i in range(len(cumulative_percentages)) if cumulative_percentages[i] > 0]

# Plotting the histogram using automatic bins
plt.figure(figsize=(10, 6))
plt.hist(total_completed_by_final, bins=bins_auto, alpha=0.7, color='blue', edgecolor='black', align='left')

plt.title('Distribution of Total Work Items Completed by the final Iteration')
plt.xlabel('Total Work Items Completed')
plt.ylabel('Frequency')
plt.show()

# Printing the list
for item in cumulative_percentages_list:
    print(item)